# Genetic Algorithms - The 8-Queens problem

Place eight queens on a 8x8 chessboard so that none of them threat each other. There are many solutions (backtracking,...). Here it is a GA approach.

Based on:

[Genetic Algorithm from Scratch in Python - Davide Anghileri](https://www.youtube.com/watch?v=c-B5mIq04bg)

### Encoding: 

Represent a solution as **string of bits**. (this is the hardest part)

Binary matrix: Simplest (usually first idea): 0s represent empty cells, 1s represent queens. Not efficient, a lot of possible arrangements (4426165368).

||a|b|c|d|e|f|g|h|
|-|-|-|-|-|-|-|-|-|
|**8**|0|0|0|0|0|1|0|0|
|**7**|0|0|0|1|0|0|0|0|
|**6**|0|0|0|0|0|0|1|0|
|**5**|1|0|0|0|0|0|0|0|
|**4**|0|0|0|0|0|0|0|1|
|**3**|0|1|0|0|0|0|0|0|
|**2**|0|0|0|0|1|0|0|0|
|**1**|0|0|1|0|0|0|0|0|

Permutation encoding (for the above matrix):

|a|b|c|d|e|f|g|h|
|-|-|-|-|-|-|-|-|
|5|3|1|7|2|8|6|4|

Stores the row of the queen in the corresponding column. More efficient, avoids invalid solutions where queens are on the same row by design (*only **40320** solutions*).



In [ ]:
# Install dependencies for implementation
%pip install sklearn-genetic
%pip install numpy

### Code Concepts

* [typing — Support for type hints](https://docs.python.org/3/library/typing.html)
* [PEP 613 – Explicit Type Aliases](https://peps.python.org/pep-0613/)
* [Partial functions](https://www.learnpython.org/en/Partial_functions)

In [10]:
# Bottom-up approach

# implement various genetic operators and then combine them to an algorithm
# def genetic_algorithm()
# population
# fitness
# loop until
#   selection ->(roulette with selection)
#   crossover
#   mutation
#   fitness
# return best

import random
from typing import TypeAlias, Callable
from functools import partial

Individual: TypeAlias =  list[int] # each individual is a possible solution
IndividualPair: TypeAlias = tuple[Individual, Individual]
Population: TypeAlias = list[Individual] # simply a list of random individuals
FitnessFunc: TypeAlias = Callable[[Individual], int]
SelectionFunc: TypeAlias = Callable[[Population, FitnessFunc], IndividualPair]
CrossoverFunc: TypeAlias = Callable[[IndividualPair], IndividualPair]
MutationFunc: TypeAlias = Callable[[Individual, float], Individual]

def generate_individual(length: int) -> Individual:
    # randomly shuffle a list of 1..8 numbers
    random.shuffle(individual := list(range(1,length + 1)))
    return individual

# pop_size is a hyperparameter (controls the optimization process), 
# should be tuned for an optimal value.
def generate_population(pop_size: int, individual_length: int) -> Population:
    return [generate_individual(individual_length) for _ in range(pop_size)]

# Fitness function: number of clashes (queens that threaten each other), 
# the lower the better = minimization problem
# How to count clashes? Permutation encoding eliminates row and column shuffles, 
# so diagonals must be counted only.
# clash: absolute difference in row position = absolute difference in column position.
def fitness(individual: Individual) -> int:
    clashes = 0
    for i in range(len(individual) - 1):
        # starts at i + 1, so we do not count the same clash twice
        for j in range(i + 1, len(individual)):
            # row == col?
            if abs(individual[j] - individual[i]) == j - i:
                clashes += 1
    return clashes

# Fitness for worst solutions(2): 28 = 7+6+5+4+3+2+1
# There are 92 optimal solutions, where fitness == 0

### Roulette with Selection

- fitter individuals have larger chance to be selected to be parents (for next generation)
- selection chance is proportional to fitness. (the idea is: good parents have even better children).

In [11]:
def roulette_selection(population: Population, fitness: FitnessFunc) -> IndividualPair:
    # Pick 2 parents with fitter parents having greater chance of being chosen:
    parents = random.choices(
        population=population,
        weights=[fitness(individual) for individual in population],
        k = 2
    )
    return parents[0], parents[1]

## Crowding:

the algorithm founds a much better solution that drowns out the other candidates. How to avoid this (increase diversity)?

- mutation: 
- switch to rank selection: (pick individual based on rank instead of fitness)
- fitness sharing: (reduce fitness of individiual if there are already similar solutions in the population)

### Crossover:


Ordered crossover: decide a random split point, take segment of Parent A before the split point, plus all the remaining elements of Parent B.

- Parent A: **731**52684
- Parent B: **8**7**26**1**4**3**5** 
- Child  X: **73182645**

In [12]:
def ordered_crossover(parents: IndividualPair) -> IndividualPair:
    parent_a, parent_b = parents
    split_idx = random.randint(1,len(parent_a) -1)
    offspring_x = parent_a[:split_idx] + list(
        filter(lambda pos: pos not in parent_a[:split_idx], parent_b)
    )
    offspring_y = parent_b[:split_idx] + list(
        filter(lambda pos: pos not in parent_b[:split_idx], parent_a)
    )
    return offspring_x, offspring_y



### Mutation:
- simply swap elements of an individual
- probability is a hyperparameter of the algorithm (tuneable)


In [13]:
def swap_mutation(individual: Individual, probability: float) -> Individual:
    if random.random() <= probability:
        pos1 = random.randint(0,len(individual) - 1)
        pos2 = random.randint(0,len(individual) - 1)
        individual[pos1], individual[pos2] = individual[pos2], individual[pos1]
    return individual

In [16]:
# how many individuals
# individual_length: 8
# fitness_limit = 0 # want an optimal solution
# mutation_prob = 
# n_iter = generations (hyperparam)
# elites = number of fittest individuals kept in the next generation (hyperparam)
def run_evolution(pop_size: int, individual_length: int, fitness_limit: int = 0, \
    mutation_prob: float = 0.3, n_elites: int = 10, n_iter: int = 1000) -> Individual:
    
    population = generate_population(pop_size, individual_length)
    for i in range(n_iter):
        population = sorted(population, key=fitness)
        print(f"Generation:{i} - best fitness:{fitness(population[0])}")
        if fitness(population[0]) <= fitness_limit:
            break
        population = compute_next_generation(population, mutation_prob, n_elites)
    return sorted(population, key=fitness)[0]

def compute_next_generation(population: Population, mutation_prob: float = 0.3,\
    n_elites: int = 10) -> Population:
    next_generation = population[:n_elites]
    for _ in range(int((len(population)  - n_elites)/ 2)):
        parents = roulette_selection(population, fitness)
        offspring = ordered_crossover(parents)
        next_generation += map(
            partial(swap_mutation, probability=mutation_prob), offspring
        )
    return next_generation

### Elitism: 

- the fittest individuals are guaranteed a place in the next generation - won't lose the best solution.

## Running the Simulation:


In [33]:
random.seed(123)
result = run_evolution(100,8)
print(result)

Generation:0 - best fitness:1
Generation:1 - best fitness:1
Generation:2 - best fitness:1
Generation:3 - best fitness:1
Generation:4 - best fitness:0
[5, 2, 4, 7, 3, 8, 6, 1]
